In [1]:
import os
os.getpid()

7747

In [2]:
import json
f = open('ESConv.json')
data = json.load(f)
f.close()
processed_data = []
for i in data:
    pt = i['problem_type']
    for j in i['dialog']:
        processed_data.append([pt, j['content'], j['speaker'], j['annotation']])

In [3]:
processed_data[4005]

['ongoing depression',
 'Sometimes we get in a funk, when we do, we can pull ourselves out, I do suggest you write a list of things to do and put on the frige.',
 'supporter',
 {'strategy': 'Others'}]

In [4]:
sl = []
for i in processed_data:
    sl.append(len(i[1].split()))
## Safely, we remove length 1,2,3
to_remove = []
for i in range(len(sl)):
    if sl[i] <= 3:
        to_remove.append(i)
for index in sorted(to_remove, reverse=True):
    del processed_data[index]

In [5]:
from collections import Counter
cpd = []
for i in processed_data:
    cpd.append(i[0])
Counter(cpd)

Counter({'job crisis': 7475,
         'problems with friends': 4702,
         'ongoing depression': 9462,
         'breakup with partner': 6697,
         'academic pressure': 4009,
         'conflict with parents': 255,
         'Procrastination': 352,
         'Alcohol Abuse': 360,
         'Issues with Parents': 239,
         'Sleep Problems': 765,
         'Appearance Anxiety': 346,
         'School Bullying': 46,
         'Issues with Children': 371})

In [6]:
issues_with_parents = []
conflict_with_parents = []
for i in processed_data:
    if i[0] == 'Issues with Parents':
        issues_with_parents.append(i[1])
    if i[0] == 'conflict with parents':
        conflict_with_parents.append(i[1])


In [7]:
issues_with_parents

['Hi, How are you?',
 'I am the youngest of 5 siblings and I feel all most days now because of my parents\n',
 "It sounds like you're having a hard time feeling like you have your parents' attention, is that right?",
 'Not their attention, but more on their expectations on me\n',
 "That sounds frustrating, I'm sorry",
 'Do they do or say anything specific that makes you feel those expectations?',
 "Yes, its more on comparing me to the children of their friends. They always say why don't I have a better job or even plans on marrying rich. Stuff like that is very frustrating",
 "They also expect me to take care of them financially soon, I'm just starting my adult life I want to enjoy it",
 'That sounds very frustrating. I think if I felt those kind of expectations I would be anxious.',
 'Do you feel comfortable talking to them about your feelings?',
 'I do feel anxious every time. ',
 'I tried a couple of times, but my mom always sees it as being too sensitive\n',
 'Sometimes it can feel

In [8]:
conflict_with_parents

['Hi, what can I help you?',
 'I am getting ready to sue my mother and am very nervous about the ordeal.',
 "You're worried about being in court against your mother or are you worried about your relationship after that?",
 'Can you tell me about the situation, why did you decide to sue her?',
 'My stepdad raped me and got me pregnant and I delivered a still born and i just remember the incident happened 23 years later. So now I am suing my mother.',
 'instead of pressing charges because my sister and I dont want it in the papers\n',
 'So she knew about the incident but did nothing to help you?',
 'she did not believe me',
 'that sucks, I understand why you resent her so much',
 'I cannot help with your worry about going to court, however, i can offer some help regarding your relationship',
 'yes it is very difficult but im not as upset as i was i cried a lot',
 "Crying is good, you shouldn't bottle your emotions",
 'Have to talked to anyone in your family about this beside your sister?

# We decided to merge 'Issues with Parents' with 'conflict with parents'

In [9]:
processed_data[0][0]

'job crisis'

In [10]:
for i in range(len(processed_data)):
    if processed_data[i][0] == 'conflict with parents':
        processed_data[i][0] = 'Issues with Parents'

In [11]:
from collections import Counter
cpd = []
for i in processed_data:
    cpd.append(i[0])
Counter(cpd)

Counter({'job crisis': 7475,
         'problems with friends': 4702,
         'ongoing depression': 9462,
         'breakup with partner': 6697,
         'academic pressure': 4009,
         'Issues with Parents': 494,
         'Procrastination': 352,
         'Alcohol Abuse': 360,
         'Sleep Problems': 765,
         'Appearance Anxiety': 346,
         'School Bullying': 46,
         'Issues with Children': 371})

In [12]:
# For now, we care the all 12 categories.
kept = ['problems with friends', 'ongoing depression','breakup with partner', 'academic pressure', 'job crisis', 'Issues with Parents', 'Procrastination', 'Alcohol Abuse','Sleep Problems','Appearance Anxiety', 'School Bullying', 'Issues with Children'   ]

filtered_processed_data = []
for i in processed_data:
    if i[0] in kept:
        filtered_processed_data.append(list(i))
        filtered_processed_data[-1][0] = kept.index(filtered_processed_data[-1][0])

# Split the training and test

In [13]:
import random

random.seed(10)
random.shuffle(filtered_processed_data)
lenth = len(filtered_processed_data)
train_list = filtered_processed_data[0:int(lenth*0.8)]
test_list = filtered_processed_data[int(lenth*0.8):]

In [14]:
# Count labels in training
lt = []
for i in train_list:
    lt.append(i[0])
Counter(lt)

Counter({0: 3785,
         1: 7533,
         2: 5361,
         3: 3162,
         4: 6018,
         7: 299,
         8: 636,
         5: 395,
         11: 302,
         9: 268,
         10: 33,
         6: 271})

In [15]:
# Count labels in testing
lts = []
for i in test_list:
    lts.append(i[0])
Counter(lts)

Counter({2: 1336,
         1: 1929,
         4: 1457,
         0: 917,
         3: 847,
         7: 61,
         9: 78,
         6: 81,
         10: 13,
         11: 69,
         8: 129,
         5: 99})

# This would be a classification into 12 categories.

In [16]:
import torch
from torch.utils.data.sampler import BatchSampler, Sampler, RandomSampler, SequentialSampler
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, BertModel, AdamW, BertConfig
from tqdm import tqdm

class Corpora():
    """
        A class that holds the texts and class labels.
    """
    
    def __init__(self, l):
        """
            file_path: on MAGIC, it is either
                        data-badassnlp/cola_public/raw/in_domain_train.tsv or
                        data-badassnlp/cola_public/raw/in_domain_dev.tsv
                    on Google Colab, change them to the corresponding paths.
        """

        self.sentences = [s[1] for s in tqdm(l)]
        self.labels = [int(s[0]) for s in tqdm(l)]
        
        
class TextClassificationDataSet(Dataset):
    """
        Define a dataset consisting of pairs of (sequence_of_word_indices, class_label).
        class_label is either 0 or 1.
    """
    def __init__(self, corpora, tokenizer: BertTokenizer):
        """
            This function will tokenize the sentences in self.corpora,
            and pad the word indices properly.
            The tokenization and padding should be done using BERT's API.

                See https://huggingface.co/docs/transformers/internal/tokenization_utils
                    for the API of tokenizer
                See https://huggingface.co/docs/transformers/preprocessing
                    for examples of using the API
                Also see the BERT paper
                    "BERT: Pre-training of Deep Bidirectional Transformers forLanguage Understanding"
                regarding how paddings are done.

            We turn the entire corpora into a big tensor in a batch, rather than
                doing that in the __getitem__ function, as batch processing can speed up tokenization
        
            Then self.input_ids, self.attention_mask and self.labels
            will be created as tensors of shape
                torch.Size([num_sentences, max_padded_sentence_length])
                torch.Size([num_sentences, max_padded_sentence_length])
                torch.Size([num_sentences])

            corpora: an object of the Corpora class representing some raw classified texts.
            tokenizer: must be the BERTTokenizer loaded from the BERT/download folder
                        in order to have the same ids for the words in any vocabulary.
                        See the loading codes below.
        """
        self.corpora = corpora
        self.tokenizer = tokenizer
        d =  tokenizer.batch_encode_plus(corpora.sentences)
        padded_encoded_inputs = tokenizer.pad(d)
        self.input_ids = torch.tensor(padded_encoded_inputs.input_ids)
        self.attention_mask = torch.tensor(padded_encoded_inputs.attention_mask)
        self.labels = torch.tensor(corpora.labels)
        
    def __len__(self):
        return len(self.corpora.sentences)

    def __getitem__(self, idx):
        """
            Return the idx-th of the rows the self.input_ids, self.attention_mask, self.labels in this order.
            Don't do BERT tokenization here as that will be slow.
        """
        return self.input_ids[idx], self.attention_mask[idx], self.labels[idx]

In [17]:
from torch import nn

class BERTClassifier(nn.Module):
    
    def __init__(self, BERT_model, hidden_layer_size, num_classes):
        
        super(BERTClassifier, self).__init__()

        # loaded pretrained model
        self.bert = BERT_model
        
        # simple neural network that convert embedding of the first token to a class
        self.classifier = nn.Sequential(
            nn.Linear(BERT_hidden_size, hidden_layer_size),
            nn.ReLU(),
            nn.Linear(hidden_layer_size, num_classes)
        )
        
    def forward(self, input_ids, attention_mask):
        """
        The following two arguments are tensors from a mini-batch of the input_ids
            and attention_mask returned by the BERT tokenizer.
            
            input_ids: a tensor of shape [batch_size, max_length]
            attention_mask: a tensor of shape [batch_size, max_length]
            
            return: the logits of the sentences in the batch tensor of shape [batch_size, 1, 2]
        """
        # see https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel
        # and https://huggingface.co/docs/transformers/main_classes/output
        # for the return value of the forward function of BERT

        ### Your codes go here (30 points) ###
        z = self.bert.forward(input_ids, attention_mask)
        return torch.softmax(self.classifier(z[1]).unsqueeze(1), dim=-1) 
        
        return logits

In [18]:
# make sure you check the availability of GPU when you set the device ID.
device = torch.device(2)

BATCH_SIZE = 8
num_classes = 12
classifier_hidden_size = 128

## if using MAGIC
BERT_PATH = '/data/badassnlp/bert-base-uncased/'
## if using Google Colab, you need to load the bert model after it downloads the model files.
## 

BERT_hidden_size = 768

N_EPOCHS = 1
CLIP = 1

In [19]:
## 'uncased' means all words are lowered-cased before tokenization
## 'base' means the smaller version of BERT (12 layers, 16 heads)
## un-comment one of the following two options.

# if using MAGIC, load from local BERT folder
tokenizer = BertTokenizer.from_pretrained(BERT_PATH, local_files_only=True)
BERT_model = BertModel.from_pretrained(BERT_PATH, num_labels = 2, output_attentions = False, output_hidden_states = False
).to(device)

## if using Colab, load from automatically downloaded files. Downloading can take half a minute
# tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
# BERT_model = BertModel.from_pretrained("bert-base-cased", num_labels = 2, output_attentions = False, output_hidden_states = False).to(device)


## if using MAGIC
print("creating training corpora ... ")
training_corpora = Corpora(train_list)
print("creating training dataset ... ")
training_dataset = TextClassificationDataSet(training_corpora, tokenizer)
print("creating training iterator ... ")
training_iterator = DataLoader(training_dataset, sampler = RandomSampler(training_dataset), batch_size=BATCH_SIZE)

## if using MAGIC
print("creating test corpora ... ")
dev_corpora = Corpora(test_list)
print("creating test dataset ... ")
dev_dataset = TextClassificationDataSet(dev_corpora, tokenizer)
print("creating test iterator ... ")
dev_iterator = DataLoader(dev_dataset, sampler = SequentialSampler(dev_dataset), batch_size=BATCH_SIZE)

classifier = BERTClassifier(BERT_model, classifier_hidden_size, num_classes).to(device)

Some weights of the model checkpoint at /data/badassnlp/bert-base-uncased/ were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


creating training corpora ... 


100%|██████████| 28063/28063 [00:00<00:00, 2382012.25it/s]

creating training dataset ... 


creating training iterator ... 
creating test corpora ... 


100%|██████████| 7016/7016 [00:00<00:00, 2381807.92it/s]

creating test dataset ... 


creating test iterator ... 


In [20]:
list(training_iterator)[0][0].shape

torch.Size([8, 198])

In [21]:
list(dev_iterator)[0][0].shape

torch.Size([8, 206])

In [22]:
from torch import optim
optimizer = optim.Adam(classifier.parameters())
criterion = nn.CrossEntropyLoss(reduction = 'sum')

# Global variable that count the number of training epoch
num_epochs_train = 0

from tqdm import tqdm

In [23]:
def train(model, iterator, optimizer, criterion, clip):
    """
        model: an BERTClassifier object
        iterator: a DataLoader object
        optimizer: torch optimizer
        criterion: the crossentropyloss
        clip: to be used with torch.nn.utils.clip_grad_norm_

        return: average loss over the training instances (sentences) in the DataLoader.
    """
    model.train()
    
    epoch_loss = 0
    num_batchs = 0
    total_instances = 0
    global num_epochs_train
    num_epochs_train += 1
    for i, batch in tqdm(enumerate(iterator)):
        num_batchs += 1
        optimizer.zero_grad() 
        tmp = optimizer.state_dict()
        tmp["param_groups"][0]["lr"] = 0.00002/(num_epochs_train)
        optimizer.load_state_dict(tmp)

        ### Your codes go here (10 points) ###

        # Step 1 (5 points): get the tensors from this mini-batch and increase the total_instances
        # make sure tensors are moved to GPU.
        ids = batch[0].to(device)
        msk = batch[1].to(device)
        y = (batch[2]).to(device)
        
        # Step 2 (5 points): call the forward function of the model and find the output logits
        # then calculate the loss. Then cumulate the epoch_loss
        logits = model.forward(ids,msk)
        loss = criterion(logits.squeeze(1), y) # if squeeze here, why unsqueeze previous
        loss.backward()

        # Clips gradient norm of an iterable of parameters.
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()
        total_instances += BATCH_SIZE

    return epoch_loss / total_instances

confusion_matrix = []
num_epochs = 0

def evaluate(model, iterator, criterion):
    """
        Find the loss (criterion) of the model over instances in this DataLoader (iterator)
        Same logic as the above train function, except no gradient calculation
        and no update of the parameter. 
        
        return: average loss over the training instances (sentences) in the DataLoader.
    """
    model.eval()

    epoch_loss = 0
    total_instances = 0
    
    confusion_matrix.append(torch.zeros(num_classes,num_classes))
    global num_epochs
    
    for i, batch in tqdm(enumerate(iterator)):
        
        ### Your codes go here (5 points) ###
        ids = batch[0].to(device)
        msk = batch[1].to(device)
        y = (batch[2]).to(device)
        logits = model.forward(ids,msk)
        loss = criterion(logits.squeeze(1), y) # if squeeze here, why unsqueeze previous
        
        epoch_loss += loss.item()
        total_instances += BATCH_SIZE
        

        for i in range(len(y)):
            row = y[i]
            col = torch.argmax((logits.squeeze(1))[i])
            confusion_matrix[num_epochs][row][col] += 1
    num_epochs += 1

    return epoch_loss / total_instances

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
import time

best_test_loss = float('inf')
training_losses = []
test_losses = []



In [39]:

for epoch in range(N_EPOCHS):  
    print("epoch start: ", epoch)  
    start_time = time.time()
    training_loss = train(classifier, training_iterator, optimizer, criterion, CLIP)
    training_losses.append(training_loss)
    test_loss = evaluate(classifier, dev_iterator, criterion)
    test_losses.append(test_loss)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if test_loss < best_test_loss:
        best_test_loss = test_loss 
        torch.save(classifier.state_dict(), 'best_model_12classes.pt')
        

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s', end='')
    print(f'\tTrain Loss: {training_loss:.3f} | Test Loss: {test_loss:.3f}')

epoch start:  0


3508it [08:39,  6.76it/s]
877it [00:36, 24.30it/s]


Epoch: 01 | Time: 9m 15s	Train Loss: 2.149 | Test Loss: 2.168


In [41]:
test_losses

[2.180055851539429, 2.1678962313326897]

In [38]:
print(confusion_matrix[0][0])
print(confusion_matrix[0][1])
print(confusion_matrix[0][2])
print(confusion_matrix[0][3])
print(confusion_matrix[0][4])
print(confusion_matrix[0][5])
print(confusion_matrix[0][6])
print(confusion_matrix[0][7])
print(confusion_matrix[0][8])
print(confusion_matrix[0][9])
print(confusion_matrix[0][10])
print(confusion_matrix[0][11])

tensor([327., 445., 102.,  10.,  33.,   0.,   0.,   0.,   0.,   0.,   0.,   0.])
tensor([ 149., 1445.,  102.,   41.,  192.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.])
tensor([151., 688., 431.,  13.,  53.,   0.,   0.,   0.,   0.,   0.,   0.,   0.])
tensor([ 42., 429.,   8., 290.,  78.,   0.,   0.,   0.,   0.,   0.,   0.,   0.])
tensor([ 61., 744.,  31.,  30., 591.,   0.,   0.,   0.,   0.,   0.,   0.,   0.])
tensor([26., 53., 15.,  2.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
tensor([ 1., 63.,  0.,  2., 15.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
tensor([ 3., 50.,  0.,  0.,  8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
tensor([  3., 105.,   6.,   2.,  13.,   0.,   0.,   0.,   0.,   0.,   0.,   0.])
tensor([ 4., 64.,  3.,  3.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
tensor([4., 8., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])
tensor([15., 45.,  3.,  4.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])


In [50]:
confusion_matrix[1][5]

tensor([18., 39., 21.,  4., 17.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

# It only recognize 5 classes.

In [51]:
# kept = ['problems with friends', 
#'ongoing depression',
#'breakup with partner', 
#'academic pressure', 
#'job crisis', 
#'Issues with Parents', 
#'Procrastination', 
#'Alcohol Abuse',
#'Sleep Problems',
#'Appearance Anxiety', 
#'School Bullying', 
#'Issues with Children'   ]
import numpy as np
sentence = "I cannot sleep last night"
es = torch.tensor([tokenizer.encode(sentence)])
msk = torch.ones(len(es[0])).unsqueeze(0)
es = es.to(device)
msk = msk.to(device)
logits = classifier.forward(es,msk)
logits

tensor([[[1.8120e-04, 9.9877e-01, 4.8491e-04, 4.2049e-05, 4.9200e-04,
          3.5026e-06, 3.1569e-06, 2.9752e-06, 5.0833e-06, 2.5254e-06,
          1.1138e-05, 4.2090e-06]]], device='cuda:2',
       grad_fn=<SoftmaxBackward0>)